In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/av-janatahack-healthcare-hackathon-ii/Data/train.csv")
train['is_train'] = 1
test = pd.read_csv("/kaggle/input/av-janatahack-healthcare-hackathon-ii/Data/test.csv")
test['is_train'] = 0
sample = pd.read_csv("/kaggle/input/av-janatahack-healthcare-hackathon-ii/Data/sample_submission_lfbv3c3.csv")


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import plotly

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold , StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
merge = pd.concat([train,test]).reset_index(drop=True)

city_code_max = max(merge["City_Code_Patient"])
bed_grade_max = max(merge["Bed Grade"])

merge["City_Code_Patient"] = merge["City_Code_Patient"].fillna(city_code_max + 1)
merge["Bed Grade"] = merge["Bed Grade"].fillna(bed_grade_max + 1)

merge["City_Code_Patient"] = merge["City_Code_Patient"].apply(lambda x: int(x))
merge["Bed Grade"] = merge["Bed Grade"].apply(lambda x: int(x))

uniques = pd.DataFrame(merge.drop(['Stay','is_train'],axis=1).nunique())
cat_columns = uniques[uniques[0] < 50].index.values.tolist()
cat_columns.remove("Available Extra Rooms in Hospital")

merge = merge[~merge['Available Extra Rooms in Hospital'].isin([14,20,24])]

for i in ['Hospital_type_code', "Hospital_region_code", "Department", "Ward_Type", 'Ward_Facility_Code', "Type of Admission", "Severity of Illness", "Age"]:
  merge[i] = pd.factorize(merge[i])[0]

#########
from tqdm import tqdm_notebook as tqdm

useful_combs = [['Ward_Type','Visitors with Patient'],
                ['Ward_Type','Bed Grade'],
                ['Bed Grade', 'Severity of Illness'],
                ['Hospital_code', 'City_Code_Patient'],
                ['City_Code_Hospital', 'City_Code_Patient'],
                ['Bed Grade', 'Type of Admission'],
                ['Severity of Illness', 'Age'],
                ['Hospital_region_code', 'Ward_Type'],
                ['Type of Admission', 'Visitors with Patient'],
                ['Ward_Type', 'Ward_Facility_Code'],
                ['Hospital_code', 'Visitors with Patient'],
                ['Type of Admission', 'Age'],
                ['Ward_Type', 'Type of Admission'],
                ['Severity of Illness', 'Visitors with Patient'],
                ['City_Code_Hospital', 'Ward_Type'],
                ['Department', 'Ward_Type'],
                ['Visitors with Patient', 'Age'],
                ['Hospital_code', 'Type of Admission'],
                ['Bed Grade', 'Age'],
                ['Department', 'Severity of Illness'],
                ['Department', 'Visitors with Patient'],
                ['City_Code_Hospital', 'Visitors with Patient'],
                ['Hospital_region_code', 'Age'],
                ['Bed Grade', 'Visitors with Patient'],
                ['Hospital_code', 'Department'],
                ['Hospital_code', 'Severity of Illness'],
                ['Hospital_region_code', 'Bed Grade']]

for comb in tqdm(useful_combs):
  one,two = comb
  name = f'{one}__{two}'
  # merge[name] = merge.apply(lambda x:str(x[one]) +"_"+str(x[two]),axis=1)
  merge[name] = merge[one].astype(str) + "_" + merge[two].astype(str)

  merge[name] = pd.factorize(merge[name])[0]  
  if len(np.unique(merge[name])) < 50:
    cat_columns.append(name)

# for f in cat_columns:
#   merge[f] = merge[f].map(merge[f].value_counts(normalize = True))
  
one,two = 'Ward_Type','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","std":f"{one}_{two}_std"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'patientid','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max"])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'Age','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'Hospital_code','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'Hospital_type_code','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'Type of Admission','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'City_Code_Patient','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'City_Code_Hospital','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = ['Hospital_code','Ward_Type'],'Visitors with Patient'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")




one,two = 'Hospital_code','Available Extra Rooms in Hospital'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'City_Code_Hospital','Available Extra Rooms in Hospital'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'Ward_Type','Available Extra Rooms in Hospital'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'Department','Available Extra Rooms in Hospital'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = 'patientid','Available Extra Rooms in Hospital'
m = merge.groupby(one)[two].agg(['mean','sum',"max"])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = 'City_Code_Patient','Available Extra Rooms in Hospital'
m = merge.groupby(one)[two].agg(['mean','sum',"max"])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = 'Ward_Type__Visitors with Patient','Available Extra Rooms in Hospital'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")





one,two = 'Hospital_code','Admission_Deposit'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = 'patientid','Admission_Deposit'
m = merge.groupby([one])[two].agg(['mean','sum',"max"])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=[one],how="left")

one,two = ['Visitors with Patient'],'Admission_Deposit'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = ['Hospital_code','Department'],'Admission_Deposit'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = 'Age','Admission_Deposit'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = 'Department','Admission_Deposit'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")

one,two = 'Ward_Type__Visitors with Patient','Admission_Deposit'
m = merge.groupby(one)[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum",
                      "std":f"{one}_{two}_std","max":f"{one}_{two}_max"})
merge = merge.merge(m,on=one,how="left")



c = ['Hospital_code','Hospital_type_code',
     'City_Code_Hospital',"Hospital_region_code",'Department',
     'Ward_Type','Type of Admission','Age']

from itertools import combinations
for comb in tqdm(combinations(c,2)):
  one,two = comb
  m = merge.groupby([one])[two].agg(['nunique','count'])
  m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
  merge = merge.merge(m,on=[one],how="left")





# one,two = 'City_Code_Hospital','patientid'
# m = merge.groupby([one])[two].agg(['nunique','count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")

# one,two = 'City_Code_Hospital','Hospital_code'
# m = merge.groupby([one])[two].agg(['nunique','count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")

# one,two = 'Hospital_code','Ward_Type'
# m = merge.groupby([one])[two].agg(['nunique','count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")

# one,two = 'Hospital_type_code','Hospital_code'
# m = merge.groupby([one])[two].agg(['count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")

# one,two = 'Age','patientid'
# m = merge.groupby([one])[two].agg(['nunique','count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")

# one,two = 'Hospital_code','patientid'
# m = merge.groupby([one])[two].agg(['nunique','count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")

# one,two = 'City_Code_Patient','patientid'
# m = merge.groupby([one])[two].agg(['nunique','count'])
# m = m.rename(columns={"nunique":f"{one}_{two}_nunique","count":f"{one}_{two}_count"})
# merge = merge.merge(m,on=[one],how="left")


#########
train = merge[merge.is_train == 1]
test = merge[merge.is_train == 0]
train.drop(["is_train",'case_id'],axis=1,inplace=True)
test.drop(["is_train",'Stay'],axis=1,inplace=True)

merge

In [ ]:
# uniques = 
# pd.DataFrame(merge.drop(['Stay','is_train'],axis=1).nunique())
test.isna().sum()
# merge.columns
# pd.DataFrame(merge.drop(['Stay','is_train'],axis=1).nunique())
# cat_columns

In [ ]:
one,two = 'Age','Visitors with Patient'
m = merge.groupby([one])[two].agg(['mean','sum',"max",'std'])
m = m.rename(columns={"mean":f"{one}_{two}_mean","sum":f"{one}_{two}_sum","min":f"{one}_{two}_min","max":f"{one}_{two}_max"})
m
# # sns.jointplot(x=one,y=two,data=train,kind="kde")
# # plt.show()
# # sns.jointplot(x=one,y=two,data=test,kind="kde")
# one,two = 'Hospital_code','Age'
# m = pd.DataFrame(merge.groupby(one)[two].value_counts()).unstack().reset_index()
# m.columns = [f"{one}_" + '_'.join([str(j) for j in i]).strip("_") + "_count" for i in m.columns]
# # merge = merge.merge(m,on=[one],how="left")
# m

In [ ]:
# col = "Available Extra Rooms in Hospital"
# train mein 14 20 24 extra hain ( Done )

# col = "patientid"

# col = "Visitors with Patient"
# train mein 25 hai lekin test mein 29

col = "Visitors with Patient"
fig,ax = plt.subplots(1,2,figsize=(18,5))

sns.countplot(sorted(train[col]),ax=ax[0])
sns.countplot(sorted(test[col]),ax=ax[1])
plt.show()

plt.figure(figsize=(9,5))
sns.countplot(sorted(merge[col]))

In [ ]:
X = train.drop(["Stay"],axis=1)
Y = train[['Stay']]

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y_ = encoder.fit_transform(Y)
Y['Stay'] = Y_

params = {
    "od_type":"Iter",
    "od_wait":150,
    "iterations":25000,
    'learning_rate':0.0224,
    "eval_metric":"Accuracy",
    "task_type":"GPU",
    # "boosting_type":"Plain"
}


def work(X,Y):
  final_preds = []
  best_score = 0
  scores = []

  folds = KFold(n_splits=5,shuffle=True,random_state=1250)

  for train_idx , test_idx in folds.split(X,Y):
    train_set = (X.iloc[train_idx],Y.iloc[train_idx])
    test_set = (X.iloc[test_idx],Y.iloc[test_idx])

    model = CatBoostClassifier(**params)
    model.fit(*train_set,
              cat_features = cat_columns,
              eval_set=[test_set],early_stopping_rounds=500,verbose=200)

    score = accuracy_score(test_set[1].values,model.predict(test_set[0]))
    print(score)
    scores.append(score)

    if score > best_score:
      best_score = score
      best_model = model

    print("---"*50)

    if final_preds == []:
      final_preds = model.predict_proba(test.drop('case_id',axis=1))
    else:
      final_preds += model.predict_proba(test.drop('case_id',axis=1))
  
  print(f"Mean Score : {np.array(scores).mean()}")
  print(f"Min Score : {np.array(scores).min()}")
  print(f"Max Score : {np.array(scores).max()}")

  plt.plot(scores)
  plt.plot(np.arange(len(scores)),[np.array(scores).mean()]*len(scores),)
  plt.show()

  final_preds /= 5
  return best_model , final_preds

model,preds = work(X,Y)
preds = encoder.inverse_transform(np.argmax(preds,axis=1))

In [ ]:
pd.set_option("display.max_colwidth",100)
pd.set_option("display.max_rows",150)
m = pd.DataFrame(list(zip(X.columns,model.feature_importances_))).sort_values(1,ascending=False)
m
# x = m[(m[1] < 1) & (m[1] > 0.8)][0].values.tolist()
# [i.split("__") for i in x if len(i.split("__")) == 2]

In [ ]:

preds

In [ ]:
# preds = model.predict(test.drop('case_id',axis=1))
# preds = model.predict(test[cols])
testing = test.copy()
testing['Stay'] = preds
x = pd.read_csv("submission.csv")
submission = testing[['case_id',"Stay"]]
submission.to_csv("submission.csv",index=False)
submission

In [ ]:
x['Stay'] = preds
x.to_csv("submission.csv",index=False)
x